In [ ]:
# Install pytorch with cuda

! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
! pip install -U transformers --quiet

  Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.0
    Uninstalling transformers-4.34.0:
      Successfully uninstalled transformers-4.34.0



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import transformers
import logging
import torch

In [2]:
def get_device():
    if torch.cuda.is_available():
        logging.info("CUDA is available.")
        return torch.device("cuda")
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        logging.info("MPS is available.")
        return torch.device("mps")
    logging.info("CUDA and MPS are not available. Using CPU.")
    return torch.device("cpu")

In [3]:
get_device()

device(type='cuda')

In [4]:
class LLModel:
    def __init__(self, model_path):
        self.model_id = model_path
        self.pipeline = transformers.pipeline(
            "text-generation",
            model=self.model_id,
            model_kwargs={
                "torch_dtype": torch.float32,
                # "quantization_config": {"load_in_4bit": True},
                # "low_cpu_mem_usage": True,
            },
        )
        self.terminators = [
            self.pipeline.tokenizer.eos_token_id,
            self.pipeline.tokenizer.convert_tokens_to_ids(""),
        ]
        # Get the EOS token ID from the tokenizerbot.chatbot()
        self.eos_token_id = self.pipeline.tokenizer.eos_token_id

    def get_response(
          self, query, message_history=[], max_tokens=4096, temperature=0.6, top_p=0.9
      ):
        user_prompt = message_history + [{"role": "user", "content": query}]
        prompt = self.pipeline.tokenizer.apply_chat_template(
            user_prompt, tokenize=False, add_generation_prompt=True
        )
        # Explicitly pass the eos_token_id to the generate function
        outputs = self.pipeline(
            prompt,
            max_new_tokens=max_tokens,
            eos_token_id=self.eos_token_id,  # Pass the eos_token_id here
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
        )
        response = outputs[0]["generated_text"][len(prompt):]
        return response, user_prompt + [{"role": "assistant", "content": response}]

    def chatbot(self, user_input="", system_instructions=""):
        conversation = [{"role": "system", "content": system_instructions}]
        response, conversation = self.get_response(user_input, conversation)
        return response

MODEL = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# I am not shure, but I think that it requires attention masks
bot = LLModel(MODEL)

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

C:\Users\dunge\Desktop\Dev Workshop\development-workshop\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dunge\.cache\huggingface\hub\models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Sliding Window Attention is enabled but not implemented for `sdp

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
bot.get_response("Why did the chicken cross the road")

('Okay, so I\'m trying to figure out why the chicken crossed the road. I know it\'s a classic riddle, but I want to understand the different ways people might answer it. Let me think through this step by step.\n\nFirst, I remember that the classic riddle is usually answered with a pun on "why" because "why" is a play on words. So, the answer is usually something like, "Because it\'s chicken," because "chicken" starts with "ch" and "why" starts with "w," but the word "chicken" has a "y" sound which makes it similar to "why." That\'s a common answer.\n\nBut I think there are other possible answers too. Maybe some people think about the chicken as a symbol of weakness or weakness in the context of crossing the road. So, they might say, "Because it\'s weak to cross the road," implying that chickens are not very strong or that they don\'t handle obstacles well. That\'s another common response.\n\nThen there\'s the idea that maybe the chicken is trying to avoid being caught by the dog. So, p

In [15]:
class TransformersModel:
    def __init__(self, model_name: str):
        self.device = get_device()
        logging.info("Loading model and tokenizer...")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        logging.info("Tokenizer loaded")
        
        # self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.model.to(self.device)
        logging.info("Model loaded")
    
    def generate_response(self, text: str, max_length=32, num_beams=4):
        inputs = self.tokenizer.encode(text, return_tensors="pt").to(self.device)
        outputs = self.model.generate(inputs, max_length=max_length, num_beams=num_beams, early_stopping=True)
        response = self.tokenizer.decode(outputs[0].to("cpu"))
        return response.replace("<pad>", "").replace("<s>", "").replace("</s>", "").strip()

models = {
    # "bigscience/T0_3B" : TransformersModel("bigscience/T0_3B"),
    # "google/flan-t5-small" : TransformersModel("google/flan-t5-small"),
    # "google/flan-t5-base" : TransformersModel("google/flan-t5-base"),
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" : TransformersModel("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"),
    # "geektech/flan-t5-base-gpt4-relation" : TransformersModel("geektech/flan-t5-base-gpt4-relation"),
}

KeyError: 'qwen2'

In [8]:
test_prompt_1 = """You are supplied an article abstract formed as a JSON. Using the provided data, create a list of age groups which the article studied. DO NOT output any other text.
                                             
                                             "abstract": {
      "Background": "The UK prevalence of abdominal aortic aneurysm (AAA) is estimated at 4.9% in over 65‐year olds. Progressive and unpredictable enlargement can lead to rupture. Endovascular repair of AAAs involves a stent graft system being introduced via the femoral artery and manipulated within the aorta under radiological guidance. Following endograft deployment, a seal is formed at the proximal and distal landing zones to exclude the aneurysm sac from the circulation. With the increasing popularity of endovascular repair there has been an increase in the number of commercially available stent graft designs on the market. This is an update of the review first published in 2013.",
      "Objectives": "This review aimed to assess the different stent graft types for endovascular repair of AAA.",
      "Search methods": "The Cochrane Vascular Group Trials Search Co‐ordinator (TSC) searched the Specialised Register (last searched February 2015) and the Cochrane Register of Studies (2015, Issue 1). Trial databases were searched by the TSC for details of ongoing and unpublished studies.",
      "Selection criteria": "All published and unpublished randomised controlled trials (RCTs) of stent graft types in the repair of AAAs were sought without language restriction and in consultation with the Cochrane Vascular Group TSC.",
      "Data collection and analysis": "We planned to conduct data collection and analysis in accordance with the Cochrane Handbook for Systematic Reviews of Interventions .",
      "Main results": "No studies were identified that met the inclusion criteria. It was not possible to review the quality of the evidence in the absence of studies eligible for inclusion in the review.",
      "Authors' conclusions": "Unfortunately, no data exist regarding direct comparisons of the performance of different stent graft types. High quality randomised controlled trials evaluating stent graft types in abdominal endovascular aneurysm repair are required."
    }
                                             """

test_prompt_2 = """You are supplied an article abstract formed as a JSON. Using the provided data, create a list of age groups which the article studied. DO NOT output any other text.
                                             
                                             "abstract": {
      "Background": "The UK prevalence of abdominal aortic aneurysm (AAA) is estimated at 4.9% in over 65‐year olds. Progressive and unpredictable enlargement can lead to rupture. Endovascular repair of AAAs involves a stent graft system being introduced via the femoral artery and manipulated within the aorta under radiological guidance. Following endograft deployment, a seal is formed at the proximal and distal landing zones to exclude the aneurysm sac from the circulation. With the increasing popularity of endovascular repair there has been an increase in the number of commercially available stent graft designs on the market. This is an update of the review first published in 2013.",
      }
                                             """

test_prompt_3 = """You are tasked with analyzing the abstract of a study. The abstract is provided below between the <abstract> HTML tags. Your goal is to extract and list the age groups mentioned in the abstract using the provided format. Do not include any explanation, additional text, or information outside the specified output format.

Format Example: "Teens 10-16 years old, Adults 20-30 years old, Elderly 65+ years old"

Abstract: <abstract>The UK prevalence of abdominal aortic aneurysm (AAA) is estimated at 4.9% in over 65‐year olds. Progressive and unpredictable enlargement can lead to rupture. Endovascular repair of AAAs involves a stent graft system being introduced via the femoral artery and manipulated within the aorta under radiological guidance. Following endograft deployment, a seal is formed at the proximal and distal landing zones to exclude the aneurysm sac from the circulation. With the increasing popularity of endovascular repair there has been an increase in the number of commercially available stent graft designs on the market. This is an update of the review first published in 2013.</abstract> """

bot.get_response(query= test_prompt_2)

# for model in models.keys():
#     print(f"Model: {model}")
#     print(f"Prompt 1 response: {models[model].generate_response(test_prompt_1, max_length=520, num_beams=4)}")
#     print(f"Prompt 2 response: {models[model].generate_response(test_prompt_2, max_length=520, num_beams=4)}")
#     print(f"Prompt 3 response: {models[model].generate_response(test_prompt_3, max_length=520, num_beams=4)}")
#     print("=====================================================================================")


('Okay, so I need to figure out how to create a list of age groups that the article studied based on the provided JSON data. The abstract is in JSON format, which I\'m somewhat familiar with, but I\'m not entirely sure how to parse it. Let me start by looking at the abstract given.\n\nThe abstract starts with a background section that mentions the prevalence of abdominal aortic aneurysms (AAA) in over 65-year-olds as 4.9%. It also talks about the increasing popularity of endovascular repair methods and the use of stent grafts. The endovascular repair involves placing a stent via the femoral artery and manipulating it within the aorta. After deployment, seals are formed at the proximal and distal landing zones to prevent the aneurysm from affecting circulation.\n\nHmm, so the main focus here seems to be on the age groups related to the prevalence of AAA and the study of endovascular repair. The background mentions "over 65-year-olds," but it doesn\'t specify further age groups. It just 

In [ ]:
prompt = f"""
  You are an expert article reviewer. You are supplied with ditailed medical article abstract. Your task is to generate very accurate PICO eligibility criteria (Population, Intervention, Comparison, Outcome).

  <article_abstract>
    <article_title>
      {record["title"]}
    </article_title>
    {sections_joined}
  </article_abstract>

  Please provide your awnser in the following format:
  <format>
    Population: (PICO eligibility criteria 1), (PICO eligibility criteria 2), ...
    Intervention: (PICO eligibility criteria 3), (PICO eligibility criteria 4), ...
    Comparison: (PICO eligibility criteria 5), (PICO eligibility criteria 6), ...
    Outcome: (PICO eligibility criteria 7), (PICO eligibility criteria 😎, ...
  </format>

  Please make the response as short and concise as possible.
  """.strip()

bot.get_response(prompt)

In [8]:
models["geektech/flan-t5-base-gpt4-relation"].generate_response("""You are an expert article reviewer. You are supplied an article abstract formed as a JSON. Using the provided data, create a comma separated list of age groups to which the article applies. This list should follow the rules of the POPULATION part of PICO eligibility criteria. Do not output any other text.
                                             
                                             "abstract": {
      "Background": "The UK prevalence of abdominal aortic aneurysm (AAA) is estimated at 4.9% in over 65‐year olds. Progressive and unpredictable enlargement can lead to rupture. Endovascular repair of AAAs involves a stent graft system being introduced via the femoral artery and manipulated within the aorta under radiological guidance. Following endograft deployment, a seal is formed at the proximal and distal landing zones to exclude the aneurysm sac from the circulation. With the increasing popularity of endovascular repair there has been an increase in the number of commercially available stent graft designs on the market. This is an update of the review first published in 2013.",
      "Objectives": "This review aimed to assess the different stent graft types for endovascular repair of AAA.",
      "Search methods": "The Cochrane Vascular Group Trials Search Co‐ordinator (TSC) searched the Specialised Register (last searched February 2015) and the Cochrane Register of Studies (2015, Issue 1). Trial databases were searched by the TSC for details of ongoing and unpublished studies.",
      "Selection criteria": "All published and unpublished randomised controlled trials (RCTs) of stent graft types in the repair of AAAs were sought without language restriction and in consultation with the Cochrane Vascular Group TSC.",
      "Data collection and analysis": "We planned to conduct data collection and analysis in accordance with the Cochrane Handbook for Systematic Reviews of Interventions .",
      "Main results": "No studies were identified that met the inclusion criteria. It was not possible to review the quality of the evidence in the absence of studies eligible for inclusion in the review.",
      "Authors' conclusions": "Unfortunately, no data exist regarding direct comparisons of the performance of different stent graft types. High quality randomised controlled trials evaluating stent graft types in abdominal endovascular aneurysm repair are required."
    }
                                             """)

KeyError: 'geektech/flan-t5-base-gpt4-relation'

In [ ]:
models["geektech/flan-t5-base-gpt4-relation"].generate_response()

'Adults 20-30 years old, Elderly 65+ years old'